In [11]:
import os
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
client_id = SPOTIPY_CLIENT_ID ='bb7c475418484e7784d9cd25b5f9f52c'
client_secret = SPOTIPY_CLIENT_SECRET='b0da0baeeab1499884912aea11f4ca58'
redirect_uri =SPOTIPY_REDIRECT_URI='https://localhost:8080/callback/'


os.environ["SPOTIPY_CLIENT_ID"] = "bb7c475418484e7784d9cd25b5f9f52c"
os.environ["SPOTIPY_CLIENT_SECRET"] = "b0da0baeeab1499884912aea11f4ca58"
os.environ["SPOTIPY_REDIRECT_URI"] = "https://localhost:8080/callback/"

scope = "playlist-modify-public playlist-modify-private user-modify-playback-state user-top-read"
scope +=            " user-modify-playback-state user-read-playback-state user-library-read user-library-modify"
username = 'tonyryanworldwide'

        
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)        
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,username=username))


In [2]:
import pandas as pd
from pandas import DataFrame
total = sp.current_user_saved_tracks()['total']
offset = 50
savedtracks = sp.current_user_saved_tracks(limit = 50)['items']
while offset < total:
    
    savedtracks2 =sp.current_user_saved_tracks(limit = 50,offset=offset)['items']
    for track in savedtracks2:
        savedtracks.append(track)
    offset += 50

    
trackinfo ={}   
for trackex in savedtracks:
    id = trackex['track']['id']
    name = trackex['track']['name']
    artistid = trackex['track']['artists'][0]['id']
    seconds = trackex['track']['duration_ms']/1000
    trackinfo[trackex['track']['id']] = {"artist" : artistid,"name":name,"seconds":seconds}
    
trackdf = pd.DataFrame.from_dict(trackinfo,orient = "index").reset_index()
artists = trackdf.artist.unique()
trackdf.columns = ['trackid','artistid','trackname','tracklength_sec']

artistinfo = []
i = 0
while i < len(artists) - 50:
    x=sp.artists(artists[i:i+50])['artists']
    artistinfo.append(x)
    i+=50
artistfinal ={}
for i in artistinfo:
    for j in i:   
        id = j['id']
        name = j['name']
        if len(j['genres']) > 0:
            genre = j['genres']#[0]
        else:
            genre = ''
        artistfinal[id] = {"name":name,"genre": genre}          
artistdf = pd.DataFrame.from_dict(artistfinal,orient = "index").reset_index()
artistdf = artistdf.reset_index()[['index','name','genre']]
artistdf.columns = ['artistid','artistname','genre']
artistdf =artistdf[artistdf['genre']!=''].reset_index()[['artistid','artistname','genre']]

In [3]:
column_names = ["artistid", "artistname", "genre"]

finalartist = pd.DataFrame(columns = column_names)

for i in range(0, len(artistdf)):
    df = artistdf[i:i+1][['artistid','artistname']]
#     print(i)
#     print(df)
    genres = artistdf[i:i+1]['genre'][i]
    gf= DataFrame(genres,columns=['genre'])
    product = (
    df.assign(key=1)
    .merge(gf.assign(key=1), on="key")
    .drop("key", axis=1)
    )
    finalartist = finalartist.append(product)
    

In [4]:
finaltrackinfo = trackdf.merge(finalartist, left_on = 'artistid', right_on = 'artistid', how = 'inner' )
finaltrackinfo['cum_sec'] = finaltrackinfo.groupby('genre')['tracklength_sec'].apply(lambda x: x.cumsum())

In [5]:
genres = finaltrackinfo.groupby('genre').count().reset_index()[['genre','artistid']]
genres.columns=['genre','count']
genres = genres[(genres['count'] >= 25) & (genres['genre']!= '')].sort_values(by ='count', ascending = False)
genrelist = genres['genre'].tolist()
finaltrackinfo = finaltrackinfo[finaltrackinfo['genre'].isin(genres['genre'])]

In [36]:
def genrefilter(df,gen,sec_lim = 14400):
    dfin = df['trackid'][(df['genre'] == gen) & (df['cum_sec'] <= sec_lim)]
    return dfin.tolist()[0:100]



In [37]:
for gen in genrelist:
    id = sp.user_playlist_create(user=username,name =gen)['id']
    tracks = genrefilter(finaltrackinfo,gen)
    sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks,position = 0)

In [20]:
playlists = sp.user_playlists(username,offset=0)['items']

[{'collaborative': False,
  'description': '',
  'external_urls': {'spotify': 'https://open.spotify.com/playlist/5YjsbzEvo0lGWq7vu1k8vk'},
  'href': 'https://api.spotify.com/v1/playlists/5YjsbzEvo0lGWq7vu1k8vk',
  'id': '5YjsbzEvo0lGWq7vu1k8vk',
  'images': [{'height': 640,
    'url': 'https://mosaic.scdn.co/640/ab67616d0000b27341df0d726f3c4c60207d72fdab67616d0000b2735901aaa980d3e714bf01171cab67616d0000b27377e8e20e0ac6182f94654a23ab67616d0000b273c9ca9a87fdd48578fc0144ec',
    'width': 640},
   {'height': 300,
    'url': 'https://mosaic.scdn.co/300/ab67616d0000b27341df0d726f3c4c60207d72fdab67616d0000b2735901aaa980d3e714bf01171cab67616d0000b27377e8e20e0ac6182f94654a23ab67616d0000b273c9ca9a87fdd48578fc0144ec',
    'width': 300},
   {'height': 60,
    'url': 'https://mosaic.scdn.co/60/ab67616d0000b27341df0d726f3c4c60207d72fdab67616d0000b2735901aaa980d3e714bf01171cab67616d0000b27377e8e20e0ac6182f94654a23ab67616d0000b273c9ca9a87fdd48578fc0144ec',
    'width': 60}],
  'name': 'hip hop',
  'ow

## For Debugging Delete CreatedPlaylists

In [30]:
# playlists =sp.current_user_playlists(offset=100)['items']
playlists = sp.user_playlists(username,offset=100)['items']
for playlist in playlists:
    name= playlist['name']
    print(name)
    
    id = playlist['uri'][-22:]
    
    if name in genrelist:
        sp.user_playlist_unfollow(user=username, playlist_id = id)

Binaural Beats  Beta Frequency
Binaural Beats 12.5Hz Alpha Frequency
Summer Song
Fly By Night
Under My Thumb
Liked from Radio
Binaural Beats 3.5Hz Delta Frequency
